# Analysis of datasets - source: MagicDraw model

Mission: verify the integrity of a system model

Objectives: 
- use pattern matching to detect the inconsistencies,
- use graph analysis to display correction suggestions,
- allocate the correction of the system model to a user.

Note: the RDF dataset shall be 
- renamed: *.trig
- available through a query endpoint

In [1]:
# import necessary libraries 
import pandas as pd
from pandas.io.json import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON

import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import Javascript

from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

import networkx as nx
import snap

In [2]:
# definition of a function: sparql query
def query_fusekiData(sparql_query, sparql_service_url):
    """
    Query the endpoint with the given query string and return the results as a pandas Dataframe.
    """
    # create the connection to the endpoint
    sparql = SPARQLWrapper(sparql_service_url)
    
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    # ask for the result
    result = sparql.query().convert()
    return json_normalize(result["results"]["bindings"])

In [3]:
# definition of ipywidget 'Text' to prompt the  endpoint to query
layout = widgets.Layout(width='auto', height='40px') #set width and height of the different widgets
endpoint=widgets.Text(
    value='http://localhost:3030/*',
    placeholder='http://localhost:3030/',
    description='Endpoint: ',
    layout = layout,
    disabled=False
)
display(endpoint)

Text(value='http://localhost:3030/*', description='Endpoint: ', layout=Layout(height='40px', width='auto'), pl…

In [4]:
# definition of ipywidget 'Button' to take into account the endpoint value prompted
output = widgets.Output()
def on_button_clicked(b):
    with output:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
        
b = widgets.Button(
    description='Analyze dataset', 
    layout = layout,
    disabled=False)
display(b)
b.on_click(on_button_clicked)

Button(description='Analyze dataset', layout=Layout(height='40px', width='auto'), style=ButtonStyle())

In [40]:
# verification of the endpoint URL
if (' ' in endpoint.value): 
    print('Error - There is a blank space in the Endpoint address.')
    raise SystemExit
else:
    req = Request(endpoint.value)
    try:
        response = urlopen(req)
    except HTTPError as e:
        print('Error - Endpoint: ' +endpoint.value + ' not found. Verify URL. (Error code: ', e.code, ')')
        raise SystemExit
    except URLError as e:
        print('Error - Server not reached. (Reason: ', e.reason, ')')
        raise SystemExit
    else:
        print ('Endpoint: ' +endpoint.value + ' found. Processing results now...')

Endpoint: http://localhost:3030/MDv20191202 found. Processing results now...


# Rule #1: Structural decomposition graph shall be directed, acyclic, connected and rooted

In [41]:
# assemblies are aggregates in assemblies
query = """
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>

SELECT DISTINCT ?physicalComponent ?isContainedIn
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a a owl:Class ;
     rdfs:label ?b ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.

  ?c a owl:Class ;
     rdfs:label ?physicalComponent ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.


  optional {?a rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:aggregates ;
               owl:someValuesFrom ?c] }.

  optional {?e rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:aggregates ;
               owl:someValuesFrom ?c ].
    ?e rdfs:label ?isContainedIn }.
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [42]:
if 'physicalComponent.value' not in result_table.columns:
    print('Error - No physical component found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[['physicalComponent.value', 'isContainedIn.value']]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

In [43]:
# from dataframe to graph
objGraph = nx.from_pandas_edgelist(obj, 'physicalComponent', 'isContainedIn', create_using=nx.DiGraph())

In [44]:
# is graph directed
print("Structural decomposition graph is directed:", nx.is_directed(objGraph))

Structural decomposition graph is directed: True


In [45]:
# is graph acyclic
print("Structural decomposition graph is acyclic:", nx.is_directed_acyclic_graph(objGraph))
if not nx.is_directed_acyclic_graph(objGraph):
    print("Number of cycles:", len(list(nx.simple_cycles(objGraph))))
    print("Cycles found:")
    for el in list(nx.simple_cycles(objGraph)):
        print('- ', el)
    print("Among them, number of self-loop edges:", objGraph.number_of_selfloops())
    print("Nodes presenting a self-loop edge:" )
    for el in objGraph.nodes_with_selfloops():
        print('- ', el)
    print("\nSuggestion: Correct the structural decomposition graph so that it becomes acyclic.")

Structural decomposition graph is acyclic: True


In [46]:
# is graph connected
print("Structural decomposition graph is connected:", nx.is_weakly_connected(objGraph))
if not nx.is_weakly_connected(objGraph):
    print("Number of isolated nodes:", len(list(nx.isolates(objGraph))))
    print("Isolated nodes:")
    for el in list(nx.isolates(objGraph)):
        print('- ', el)
    print("\nSuggestion: Connect the isolated nodes.")

Structural decomposition graph is connected: True


In [47]:
# is graph rooted
obj = pd.DataFrame(objGraph.out_degree())
obj = obj[obj[1]==0]
print("Structural decomposition graph is rooted:", obj.shape[0] == 1)
if obj.shape[0] == 0:
    print("No roots found.")
    print("\nSuggestion: Correct the structural decomposition graph so that it presents one and only one root.")
if obj.shape[0] > 1:
    print("Number of roots found:", obj.shape[0])
    print("Roots found:")
    for el in obj[0]:
        print('-', el)
    print("\nSuggestion: Correct the structural decomposition graph so that it presents one and only one root.")

Structural decomposition graph is rooted: True


# Rule #2: Each Physical Component shall be contained in exactly one Subsystem

In [48]:
# physical components are contained in subsystems sparql query
query = """
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>

SELECT DISTINCT ?physicalComponent ?subSystem
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a a owl:Class ;
     rdfs:label ?b ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.

  ?c a owl:Class ;
     rdfs:label ?physicalComponent ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.


  optional {?a rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:contains ;
               owl:someValuesFrom ?c] }.

  optional {?e rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:contains ;
               owl:someValuesFrom ?c ].
    ?e rdfs:label ?subSystem }.
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [49]:
if 'subSystem.value' not in result_table.columns:
    print('Error - No subsystem found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[['physicalComponent.value', 'subSystem.value']]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

In [50]:
# from dataframe to graph
objGraph = nx.from_pandas_edgelist(obj, 'physicalComponent', 'subSystem', create_using=nx.DiGraph())

### a/ Is there Physical Components not contained in any Subystem?

In [51]:
sol_a = obj.subSystem.isnull().sum() # select rows based on condition 

print('Number of physical components not aggregated in any subsystem: '+str(sol_a))
sol_a = obj[obj['subSystem'].isnull()] 
sol_a = sol_a['physicalComponent'].tolist()

if not sol_a:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of physical component currently not aggregated in any subsystem: ')
    for el in sol_a:
        print ("- " +el)
    print("\nSuggestion: Connect each of these physical components to one subsystem.")

Number of physical components not aggregated in any subsystem: 0
The model complies with the rule. No action needs to be performed.


### b/ Is there any Physical Components contained in more than one Subsystem?

In [52]:
sol_b = obj.groupby(["physicalComponent"]).size()
sol_b = sol_b.to_frame("subSystem_count")

sol_b = sol_b[sol_b['subSystem_count'] > 1] # select rows based on condition 
print('Number of physical components aggregated in more than one subsystem: '+str(sol_b.shape[0]))

if sol_b.empty:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of physical components currently aggregated in more than one subsystem: ')
    for i in range(sol_b.shape[0]):
        print('- Physical component ' +list(sol_b.index)[i] +' is aggregated in ' +str(list(sol_b.subSystem_count)[i]) +' different Subsystems: ')
        for j in range((obj[obj['physicalComponent'] == list(sol_b.index)[i]]).shape[0]):
            print('  o ' +list(obj[obj['physicalComponent'] == list(sol_b.index)[i]].subSystem)[j])
    print("\nSuggestion: Look for these physical components and delete the extra base:contains relationships.")

Number of physical components aggregated in more than one subsystem: 0
The model complies with the rule. No action needs to be performed.


# Rule #3: Each Subsystem shall be supplied by exactly one WorkPackage

In [53]:
# workpackages supply subsystems sparql query
query = """
PREFIX project: <http://imce.jpl.nasa.gov/foundation/project/project#>
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?subsystem ?workPackage

FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#Subsystem_ID__18_5_3_8c20287_1572979614280_430250_25878>;
     rdfs:label ?subsystem.
  
  optional {
    ?b rdfs:subClassOf  [ a owl:Restriction ; owl:onProperty project:supplies ; owl:someValuesFrom ?a ] ;
                                                                                rdfs:label ?workPackage. }  
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [54]:
if 'subsystem.value' not in result_table.columns:
    print('Error - No subsystem found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[["subsystem.value", "workPackage.value"]]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

### a/ Is there Subsystems not supplied by any WorkPackage?

In [55]:
sol_a = obj.workPackage.isnull().sum() # select rows based on condition 

print('Number of subsystems not supplied by any workpackage: '+str(sol_a))
sol_a = obj[obj['workPackage'].isnull()] 
sol_a = sol_a['subsystem'].tolist()

if not sol_a:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of subsystems currently not supplied by any workpackage: ')
    for el in sol_a:
        print ("- " +el)
    print("\nSuggestion: Connect each of these subsystems to one workpackage.")

Number of subsystems not supplied by any workpackage: 0
The model complies with the rule. No action needs to be performed.


### b/ Is there Subsystems supplied by more than one WorkPackage?

In [56]:
sol_b = obj.groupby(["subsystem"]).size()
sol_b = sol_b.to_frame("workPackage_count")

sol_b = sol_b[sol_b['workPackage_count'] > 1] # select rows based on condition 
print('Number of subsystems supplied by more than one workpackage: '+str(sol_b.shape[0]))

if sol_b.empty:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of subsystems currently supplied by more than one workpackage: ')
    for i in range(sol_b.shape[0]):
        print('- Subsystem ' +list(sol_b.index)[i] +' is supplied by ' +str(list(sol_b.workPackage_count)[i]) +' different WorkPackages: ')
        for j in range((obj[obj['subsystem'] == list(sol_b.index)[i]]).shape[0]):
            print('  o ' +list(obj[obj['subsystem'] == list(sol_b.index)[i]].workPackage)[j])
    print("\nSuggestion: Look for these subsytems and delete the extra project:supplies relationships.")

Number of subsystems supplied by more than one workpackage: 0
The model complies with the rule. No action needs to be performed.


# Rule #4: Each Physical Component shall perform at least one Function

In [57]:
# assemblies perform functions sparql query
query = """
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>
PREFIX mission: <http://imce.jpl.nasa.gov/foundation/mission/mission#>

SELECT DISTINCT ?physicalComponent ?function
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a a owl:Class ;
     rdfs:label ?physicalComponent ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.

  optional {
    ?a rdfs:subClassOf  [ a owl:Restriction ; owl:onProperty mission:performs ; owl:someValuesFrom ?b ].
    ?b rdfs:label ?function
    }
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [58]:
if 'physicalComponent.value' not in result_table.columns:
    print('Error - No physical component found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[['physicalComponent.value', 'function.value']]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

In [59]:
sol_a = obj.function.isnull().sum() # select rows based on condition 

print('Number of physical components not performing any function: '+str(sol_a))
sol_a = obj[obj['function'].isnull()] 
sol_a = sol_a['physicalComponent'].tolist()

if not sol_a:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of physical components currently not performing any function: ')
    for el in sol_a:
        print ("- " +el)
    print("\nSuggestion: Connect each of these physical components to exactly one function.")

Number of physical components not performing any function: 7

List of physical components currently not performing any function: 
- Launch vehicle interface module
- Avionics stack section
- Propellant handling section
- Base plate module
- Top panel module
- Propulsion module
- Payload module

Suggestion: Connect each of these physical components to exactly one function.


# Rule #5: Each Physical Component shall be specified by at least one Requirement

In [60]:
# assemblies are specified by requirements sparql query
query = """
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX mission: <http://imce.jpl.nasa.gov/foundation/mission/mission#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>

SELECT DISTINCT ?physicalComponent ?requirement
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a a owl:Class ;
     rdfs:label ?physicalComponent ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.

  optional {
    ?b rdfs:subClassOf  mission:Requirement ;
       rdfs:label ?requirement.
    ?b rdfs:subClassOf [ a owl:Restriction ; owl:onProperty mission:specifies ; owl:someValuesFrom ?a ].  
  }
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [61]:
if 'physicalComponent.value' not in result_table.columns:
    print('Error - No physical component found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[['physicalComponent.value','requirement.value']]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

In [62]:
sol = obj.requirement.isnull().sum() # select rows based on condition 

print('Number of physical components not specified by any requirement: '+str(sol))
sol = obj[obj['requirement'].isnull()] 
sol = sol['physicalComponent'].tolist()

if not sol:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of physical components currently not specified by any requirement: ')
    for el in sol:
        print ("- " +el)
    print("\nSuggestion: Connect each of these physical components to at least one requirement.")

Number of physical components not specified by any requirement: 7

List of physical components currently not specified by any requirement: 
- Launch vehicle interface module
- GPS receiver unit
- Avionics stack section
- Propellant handling section
- Base plate module
- Top panel module
- Payload module

Suggestion: Connect each of these physical components to at least one requirement.


# Rule #6: Each Function shall be performed by exactly one Physical Component

In [63]:
# functions are performed by assemblies sparql query
query = """
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX mission: <http://imce.jpl.nasa.gov/foundation/mission/mission#>

SELECT DISTINCT ?function ?physicalComponent 
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a rdfs:subClassOf  [ a owl:Restriction ; owl:onProperty mission:performs ; owl:someValuesFrom ?b ].
  ?b rdfs:label ?function

  optional {
    ?c a owl:Class ;
       rdfs:label ?physicalComponent ;
       rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.
    ?c rdfs:subClassOf [ a owl:Restriction ; owl:onProperty mission:performs ; owl:someValuesFrom ?b ]. 
  }
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [64]:
if 'function.value' not in result_table.columns:
    print('Error - No function found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[['function.value', 'physicalComponent.value']]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

### a/ Is there Functions not performed by any Physical Component?

In [65]:
sol_a = obj.physicalComponent.isnull().sum() # select rows based on condition 

print('Number of functions not performed by any physical component: '+str(sol_a))
sol_a = obj[obj['physicalComponent'].isnull()] 
sol_a = sol_a['function'].tolist()

if not sol_a:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of functions currently not performed by any physical component: ')
    for el in sol_a:
        print ("- " +el)
    print("\nSuggestion: Connect each of these functions to at least one physical component.")

Number of functions not performed by any physical component: 0
The model complies with the rule. No action needs to be performed.


### b/ Is there Functions performed by more than one Physical Component?

In [66]:
sol_b = obj.groupby(["function"]).size()
sol_b = sol_b.to_frame("physicalComponent_count")

sol_b = sol_b[sol_b['physicalComponent_count'] > 1] # select rows based on condition 
print('Number of functions performed by more than one physical component: '+str(sol_b.shape[0]))

if sol_b.empty:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of functions currently performed by more than one physical component: ')
    for i in range(sol_b.shape[0]):
        print('- Function ' +list(sol_b.index)[i] +' is performed by ' +str(list(sol_b.physicalComponent_count)[i]) +' different Physical Components: ')
        for j in range((obj[obj['function'] == list(sol_b.index)[i]]).shape[0]):
            print('  o ' +list(obj[obj['function'] == list(sol_b.index)[i]].physicalComponent)[j])
    print("\nSuggestion: Look for these functions and delete the extra mission:performs relationships.")

Number of functions performed by more than one physical component: 14

List of functions currently performed by more than one physical component: 
- Function Apply commands is performed by 2 different Physical Components: 
  o Primary flight computer unit
  o Receiver unit
- Function Attitude awareness is performed by 3 different Physical Components: 
  o Magnetometer
  o Sun sensor
  o Earth sensor
- Function Control trajectory is performed by 20 different Physical Components: 
  o Magnetorquer 3
  o Reaction wheel 1
  o Magnetorquer 2
  o Reaction wheel 3
  o ACS thruster 5
  o ACS thruster 2
  o Magnetorquer 1
  o Reaction wheel 4
  o ACS thruster 3
  o Propellant tank
  o ACS thruster 1
  o Propulsion system control unit
  o BP propellant line unit
  o ACS thruster 6
  o Reaction wheel 2
  o Fill/Drain valve unit
  o ACS thruster 4
  o ADCS electronics unit
  o Pressurant tank
  o PM propellant line unit
- Function Deploy mechanisms is performed by 5 different Physical Components: 

# Rule #7: Each Requirement shall specify at least one Physical Component or a mission:performs association¶

In [67]:
# requirements specify assemblies sparql query
query = """
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX mission: <http://imce.jpl.nasa.gov/foundation/mission/mission#>

SELECT DISTINCT ?requirement ?physicalComponent ?fonctionalComponent 
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a rdfs:subClassOf  mission:Requirement ;
     rdfs:label ?requirement.

  optional {
    ?c a owl:Class ;
       rdfs:label ?physicalComponent ;
       rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.
    ?a rdfs:subClassOf [ a owl:Restriction ; owl:onProperty mission:specifies ; owl:someValuesFrom ?c ].  
  }
  optional{
    ?e a owl:Class ;
       rdfs:label ?fonctionalComponent ;
       rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.
    ?a rdfs:subClassOf [ a owl:Restriction ; owl:onProperty mission:specifies ; owl:someValuesFrom ?x ]. 

    ?x rdfs:subClassOf [ a owl:Restriction ; owl:onProperty mission:hasPerformsSource ; owl:someValuesFrom ?e ].
  }
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [68]:
if 'requirement.value' not in result_table.columns:
    print('Error - No requirement found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[['requirement.value', 'physicalComponent.value', 'fonctionalComponent.value']]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

In [69]:
sol = obj.physicalComponent.isnull() & obj.fonctionalComponent.isnull() # select rows based on condition 
sol = list(sol)

print('Number of requirements that do not specify any physical component nor any mission:performs association: '+str(sol.count(True)))
sol = obj[(obj['physicalComponent'].isnull()) & (obj['fonctionalComponent'].isnull())]
sol = sol['requirement'].tolist()

if not sol:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of requirements that currently do not specify any physical component nor any mission:performs association: ')
    for el in sol:
        print ("- " +el)
    print("\nSuggestion: Connect each of these requirements to at least one physical component or to one mission:performs association.")

Number of requirements that do not specify any physical component nor any mission:performs association: 3

List of requirements that currently do not specify any physical component nor any mission:performs association: 
- Time allocation for ground receipt and confirmation OUTSIDE SCOPE
- Timeliness
- Timeliness-refined

Suggestion: Connect each of these requirements to at least one physical component or to one mission:performs association.


# Rule #8: Each WorkPackage shall be under the responsability of exactly one organization

In [70]:
# workPackages under the responsability of organizations sparql query
query = """
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX project: <http://imce.jpl.nasa.gov/foundation/project/project#>

SELECT DISTINCT ?workPackage ?organization 
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a rdfs:subClassOf  project:WorkPackage ;
     rdfs:label ?workPackage.
  
  optional {
  ?b rdfs:subClassOf project:Organization ;
     rdfs:label ?organization ;
     rdfs:subClassOf  [ a owl:Restriction ; owl:onProperty project:hasResponsibilityFor ; owl:someValuesFrom ?a ] . }
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [71]:
if 'workPackage.value' not in result_table.columns:
    print('Error - No workPackage found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[['workPackage.value', 'organization.value']]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

### a/ Is there WorkPackages not under the responsability of any organization?

In [72]:
sol_a = obj.organization.isnull().sum() # select rows based on condition 

print('Number of workPackages not under the responsability of any organization: '+str(sol_a))
sol_a = obj[obj['organization'].isnull()] 
sol_a = sol_a['workPackage'].tolist()

if not sol_a:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of workPackages currently not under the responsability of any organization: ')
    for el in sol_a:
        print ("- " +el)
    print("\nSuggestion: Connect each of these workPackages to exactly one organization.")

Number of workPackages not under the responsability of any organization: 0
The model complies with the rule. No action needs to be performed.


### b/ Is there WorkPackages under the responsability of more than one organization?

In [73]:
sol_b = obj.groupby(["workPackage"]).size()
sol_b = sol_b.to_frame("organization_count")

sol_b = sol_b[sol_b['organization_count'] > 1] # select rows based on condition 
print('Number of workPackages under the responsability of more than one organization: '+str(sol_b.shape[0]))

if sol_b.empty:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of workPackages under the responsability of more than one organization: ')
    for i in range(sol_b.shape[0]):
        print('- WorkPackage ' +list(sol_b.index)[i] +' is under the responsability of ' +str(list(sol_b.organization_count)[i]) +' different Organizations: ')
        for j in range((obj[obj['workPackage'] == list(sol_b.index)[i]]).shape[0]):
            print('  o ' +list(obj[obj['workPackage'] == list(sol_b.index)[i]].organization)[j])
    print("\nSuggestion: Look for these workPackages and delete the extra project:hasResponsibilityFor relationships.")

Number of workPackages under the responsability of more than one organization: 0
The model complies with the rule. No action needs to be performed.


# Rule #9: Each power load assembly shall be connected to at least one power source assembly

In [74]:
print("To be implemented when interfaces will be in the OWL data.")

To be implemented when interfaces will be in the OWL data.


# Rule #10: Each power load end circuit interface shall be connected to one power source end circuit interface

In [75]:
print("To be implemented when interfaces will be in the OWL data.")

To be implemented when interfaces will be in the OWL data.


# Rule #11: The Function decomposition graph shall be directed and acyclic

# Rule #12: The Requirement decomposition graph shall be directed and acyclic